In [0]:
import pandas as pd, numpy as np
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"

pd.options.display.max_columns = None
pd.options.display.max_rows = 30

In [0]:
def converter(dfdf):
    items = ['ITM_ID', 'GTIN', 'PMY_DPT_CD', 'REC_DPT_CD', 'DPT_CD', 'COM_CD', 'SUBCOM_CD', 'L1_Scores', 'L1 Scores', 'L2_Scores', 'L2 Scores', 'Score', 'L2_Flag', 'L2 Flag', 'Scores', 'Threshold', 'First Scores', 'Second Scores', 'Third Scores', 'Probability Scores', 'L1 Predictions', 'Level 1 Predictions']
    for i in items:
        if i in dfdf.columns.tolist():
            dfdf[i] = dfdf[i].astype(np.float64)
    return dfdf

In [0]:
pim_gtin_mapped = pd.read_parquet(DBFR + "PIM_DATA_NEW_51LAKH.parquet")

groc_com = pd.read_csv(DBFM + "GROCERY_COMs_for_Flavor_Attribution.csv")
flav_attb = pd.read_csv(DBFM + "PIM_DATA_WITH_FLAVOR_ATTB_100323.csv")
flav_attb = converter(flav_attb)

In [0]:
pim_gtin_mapped = pd.read_csv(DBFR + "PIM_DATA_NEW_51LAKH.zip")
pim_gtin_mapped  =converter(pim_gtin_mapped)

COMs1 = groc_com[groc_com['Has Flavor'].isna()]['COM_DSC']
COMs = pd.read_csv(DBFO + "COM_CD_File.csv")
COMs = COMs['COM_CD'].astype(np.float64)

import numpy as np
pim_gtin_mapped['Flag_COM_CD'] = np.nan
pim_gtin_mapped['Flag_Groc'] = np.nan
for i in pim_gtin_mapped[pim_gtin_mapped['COM_CD'].isin(COMs)].index.tolist():
    pim_gtin_mapped.at[i, 'Flag_COM_CD'] = 1
for i in pim_gtin_mapped[pim_gtin_mapped['COM_DSC'].isin(COMs1.tolist())].index.tolist():
    pim_gtin_mapped.at[i, 'Flag_Groc'] = 1


pim_gtin_mapped = converter(pim_gtin_mapped)
flav_attb = converter(flav_attb)

pim_gtin_mapped = pim_gtin_mapped.merge(flav_attb[['GTIN', 'Flavor_Code']], on=['GTIN'], how='left')

In [0]:
flav_notnull = pim_gtin_mapped[pim_gtin_mapped['Flavor_Code'].notnull()]
flav_null = pim_gtin_mapped[pim_gtin_mapped['Flavor_Code'].isna()]

flav_notnull.drop(index=flav_notnull[flav_notnull['Flavor_Code'].isin(['MISCELLANEOUS', 'OTHER', 'NONE'])].index, inplace=True)
flav_notnull['Flavor_Code'].value_counts()

nulll = flav_null[flav_null.SUBCOM_CD.isin(flav_notnull['SUBCOM_CD'].unique().tolist())]

In [0]:
from rapidfuzz import fuzz, process
def fuzzToken(ONE,TWO):
    best = process.extract(
       str(ONE), [str(x) for x in TWO], scorer=fuzz.token_set_ratio #, score_cutoff=10
    )
    return best

In [0]:
IC_UNIQUE_FC = ['PECAN', 'VANILLA', 'CARAMEL', 'PINEAPPLE', 'CHOCOLATE',
'FRENCH_VANILLA', 'CHOCOLATE_CHIP', 'PEANUT_BUTTER', 'BUTTERMILK',
'PUMPKIN_PIE', 'STRAWBERRY', 'CHERRY', 'COFFEE',
'BROWNIE', 'MINT', 'TOFFEE', 'SALTED', 'RED_VELVET', 'LEMON',
'PEPPERMINT', 'CHOCOLATE_MINT', 'GREEN_TEA', 'WHITE_CHOCOLATE',
'BUTTER_PECAN', 'ORANGE', 'DARK_CHOCOLATE', 'CINNAMON', 'FRUIT',
'ALMOND', 'BANANA', 'GINGER', 'BLUEBERRY', 'PEACH', 'MANGO',
'BUTTER', 'BUTTERSCOTCH', 'COCOA', 'RASPBERRY', 'MOCHA', 'HONEY',
'POPPY_SEED', 'BOURBON', 'BUTTER_TOFFEE', 'LIME', 'TRUFFLE',
'SUGAR_SPICE', 'PUMPKIN', 'BIRTHDAY_CAKE', 'EGG', 'OATMEAL',
'APPLE', 'WALNUT', 'MARSHMALLOW', 'PLAIN', 'COCONUT', 'PISTACHIO',
'COTTON_CANDY', 'MAPLE', 'BERRY', 'PASSION_FRUIT', 'POMEGRANATE']

In [0]:
BB = pim_gtin_mapped[pim_gtin_mapped['COM_DSC'].isin(['ICE CREAM ICE MILK & SHERBETS', 'NF FROZEN ICE CREAM','FOOD SERVICE-ICE CREAM', 'NF FRZ ICE CREAM & NOVELTIES'])]
BB = BB.reset_index().drop(columns=['index'])

In [0]:
FINAL = pd.DataFrame()
BB = BB[~BB.VND_ECOM_DSC.isna()]

for i in range(len(BB.VND_ECOM_DSC)):
    Temp = fuzzToken(BB.VND_ECOM_DSC.iloc[i].upper(), IC_UNIQUE_FC)
    ANS = pd.DataFrame([i[0:2] + i[3:5] + i[6:8] + i[9:11] + i[12:14] for i in Temp]).T
    ANS = ANS.unstack().to_frame().T
    temp = BB.iloc[i]
    data = {
        'GTIN': temp['GTIN'],
        'ITM_ID': temp['ITM_ID'],
        'PMY_DPT_CD' : temp['PMY_DPT_CD'],
        'PMY_DPT_DSC' : temp['PMY_DPT_DSC'],
        'REC_DPT_CD' : temp['REC_DPT_CD'],
        'REC_DPT_DSC' : temp['REC_DPT_DSC'],
        'DPT_CD' : temp['DPT_CD'],
        'DPT_DSC' : temp['DPT_DSC'],
        'COM_CD' : temp['COM_CD'],
        'COM_DSC' : temp['COM_DSC'],
        'SUBCOM_CD' : temp['SUBCOM_CD'],
        'SUBCOM_DSC' : temp['SUBCOM_DSC'],
        'VND_ECOM_DSC' : temp['VND_ECOM_DSC']
    }
    FINAL = pd.concat([FINAL,
                       pd.concat([pd.DataFrame(data, index=[i]), pd.DataFrame(ANS.values, index=[i])], axis=1)],
                       ignore_index=True)

FINAL.rename(columns={0:'Flavor_Code1', 1:'Scores 1',2:'Flavor_Code2',3:'Scores 2',4:'Flavor_Code3',5:'Scores 3',6:'Flavor_Code4',7:'Scores 4',8:'Flavor_Code5',9:'Scores 5'}, inplace=True)

In [0]:
FINAL